In [ ]:
plt.figure(figsize=(12,6))
ax = plt.subplot(121)
plot_polygons(ax, election.geometry)

election_ea = election.to_crs(epsg=5070)
ax = plt.subplot(122)
plot_polygons(ax, election_ea.geometry)

In [ ]:
election48 = election_ea[~election_ea['state'].isin(['AK', 'HI'])]
election48.to_file('election-counties-48.geojson', driver='GeoJSON')
election48 = gp.read_file('election-counties-48.geojson')

In [ ]:
plt.figure(figsize=(16,8))
ax = plt.subplot(121)
ax.set_title('Democratic votes')
plot_polygons(ax, election48.geometry, values=election48.dem, colormap="Blues")
ax = plt.subplot(122)
ax.set_title('Republican votes')
plot_polygons(ax, election48.geometry, values=election48.gop, colormap="Reds")

In [ ]:
election48['votes'] = election48.dem + election48.gop + election48.grn + election48.lib + election48.una + election48.oth
election48['goppc'] = election48.gop / election48.votes * 100
election48['dempc'] = election48.dem / election48.votes * 100

In [ ]:
plt.figure(figsize=(12,9))
ax = plt.subplot(111)
ax.set_title('Republican share of votes')
plot_polygons(ax, election48.geometry, values=election48.goppc, colormap="RdBu_r")

In [ ]:
states = election_ea.dissolve(by='state', aggfunc='sum', as_index=False)
states['votes'] = states.dem + states.gop + states.grn + states.lib + states.una + states.oth
states['dempc'] = states.dem / states.votes
states['goppc'] = states.gop / states.votes

In [ ]:
states.head()

In [ ]:
states.plot(column="goppc", cmap="Reds", figsize=(12,6), linewidth=0.1)

In [ ]:
def get_max_idx(L):
    max_i = 0
    for i in range(len(L)):
        if L[i] >= L[max_i]:
            max_i = i
    return max_i

def remove_i(L, i):
    return L[:i] + L[i+1:]

def insert_i(L, i, x):
    return L[:i] + [x] + L[i:]

def apportion(pops, states, seats_to_assign=435, initial=1, extras=2, exclude='DC'):
    pops = list(pops)
    states = list(states)
    assigned = [initial] * len(pops)
    ex = states.index(exclude)
    assigned = remove_i(assigned, ex)
    pops = remove_i(pops, ex)
    remaining = seats_to_assign - sum(assigned)
    while remaining > 0:
        priorities = [p / np.sqrt(a * (a + 1)) for p, a in zip(pops, assigned)]
        max_priority = get_max_idx(priorities)
        assigned[max_priority] += 1
        remaining -= 1
    assigned = insert_i(assigned, ex, 1)
    assigned = [__ + 2 for __ in assigned]
    return assigned

In [ ]:
states['ev'] = apportion(states.population, states.state)
states.head()

In [ ]:
import pysal as ps
ps.version
neighbors = ps.weights.Contiguity.Rook.from_dataframe(election_ea)

In [ ]:
neighbors[0]

In [ ]:
nss = []
for i, Ni in neighbors: 
    Ns = {}
    for j, w in Ni.items():
        Ns[j] = election_ea.state[j]
    nss.append((i, election_ea.state[i], Ns))
nss[:5]

In [ ]:
boundary_pairs = []
boundary_cases = set()
for Ni in nss:
    i, i_s, Nj = Ni[0], Ni[1], Ni[2]
    for j, j_s in Nj.items():
        if i_s != j_s:
            boundary_pairs.append((i, j))
            boundary_cases = boundary_cases.union([i, j])


In [ ]:
election['boundary'] = election.index.isin(boundary_cases)

In [ ]:
fig, ax = plt.subplots(figsize=(12,9))
ax.set_aspect('equal')
plot_polygons(ax=ax, geoms=election_ea.geometry)
plot_polygons(ax=ax, geoms=election_ea.geometry[np.where(election_ea.boundary)], facecolor='white')